# Работа с API YandexGPT

In [9]:
import requests
import json
import time

Для работы с API YandexGPT необходим API-ключ для сервисного аккаунта Яндекс.Облака с ролями 'ai.languageModels.user' и 'ai.imageGeneration.user'.

Подробнее о создании API-ключа: https://yandex.cloud/ru/docs/iam/concepts/authorization/api-key

In [10]:
yandex_cloud_catalog = ""
yandex_api_key = ""

Варианты модели:
- yandexgpt-lite
- yandexgpt
- summarization (для суммарищации текстов)

In [11]:
yandex_gpt_model = "yandexgpt-lite"

Структура промта:
- Общая инструкция (Ты ...)
- Запрос (Напиши ...)
- Контекст (Текст)
- Формат ответа

Параметры генерации
- temperature: чем выше значение этого параметра, тем более креативными и случайными будут ответы модели. Принимает значения от 0 до 1

In [16]:
temperature = 0.6
prompt = "Ты поэт и автор стихов для детей. Придумай стих про программиста из 4 строк. В ответе напиши только стих."

In [21]:
body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {
        "stream": False,
        "temperature": temperature,
        "maxTokens": "2000",
    },
    "messages": [{"role": "user", "text": prompt}],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_api_key}",
}

response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
answer = response_json["result"]["alternatives"][0]["message"]["text"]
print(answer)

Программист за кодом день-деньской,
Ловко управляет он строкой.
Строки те – команды для программ,
Нужен программист всегда и нам!


### Добавления системного промпта

In [22]:
system_prompt = "Исправь ошибки в тексте. В ответе напиши только исправленный текст."
prompt = "Сабака гуляет по трове а люди набблюдают"

In [23]:
import requests
import json

body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {"stream": False, "temperature": 0, "maxTokens": "2000"},
    "messages": [
        {"role": "system", "text": system_prompt},
        {"role": "user", "text": prompt},
    ],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_gpt_api_key}",
    "x-folder-id": yandex_cloud_catalog,
}

response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
answer = response_json["result"]["alternatives"][0]["message"]["text"]
print(answer)

Собака гуляет по траве, а люди наблюдают.


### Работа в асинхронном режиме

Выполнение запроса на генерацию текста в асинхронном режиме. Генерация может выполняться несколько секунд. В результате выполнения запроса получаем operation_id, с помощью которого можно получать результат, выполнив отдельный запрос

In [25]:
body = {
    "modelUri": f"gpt://{yandex_cloud_catalog}/{yandex_gpt_model}",
    "completionOptions": {"stream": False, "temperature": 0, "maxTokens": "2000"},
    "messages": [
        {"role": "system", "text": system_prompt},
        {"role": "user", "text": prompt},
    ],
}

url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completionAsync"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Api-Key {yandex_api_key}",
    "x-folder-id": yandex_cloud_catalog,
}

response = requests.post(url, headers=headers, json=body)
response_json = json.loads(response.text)
print(response_json)
operation_id = response_json["id"]

{'id': 'd7qff3heaq86t7v8r6e5', 'description': 'Async GPT Completion', 'createdAt': '2024-04-14T04:30:24Z', 'createdBy': 'aje4pspgk8umd6h29st2', 'modifiedAt': '2024-04-14T04:30:24Z', 'done': False, 'metadata': None}


Необходимо дождаться выполнения генерации

In [27]:
url = f"https://llm.api.cloud.yandex.net/operations/{operation_id}"
headers = {"Authorization": f"Api-Key {yandex_api_key}"}

while True:
    response = requests.get(url, headers=headers)
    response_json = json.loads(response.text)
    done = response_json["done"]
    if done:
        break
    else:
        time.sleep(0.5)

answer = response_json["response"]["alternatives"][0]["message"]["text"]
print(answer)

Собака гуляет по траве, а люди наблюдают.
